In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import GPy
from dipy.core.gradients import gradient_table
from diGP.preprocessing_pipelines import preprocess_SPARC
from diGP.dataManipulations import (DataHandler, log_q_squared)
from diGP.model import Model

%matplotlib inline
sns.set_style('dark')

In [ ]:
dataPath = {'SPARC_20': 'C:\\Users\\sesjojen\\Documents\\Data\\SPARC\\nifti\\gradient_20_nifti',
            'SPARC_30': 'C:\\Users\\sesjojen\\Documents\\Data\\SPARC\\nifti\\gradient_30_nifti',
            'SPARC_60': 'C:\\Users\\sesjojen\\Documents\\Data\\SPARC\\nifti\\gradient_60_nifti',
            'SPARC_GS': 'C:\\Users\\sesjojen\\Documents\\Data\\SPARC\\nifti\\goldstandard_nifti'}
q_test_path = 'C:\\Users\\sesjojen\\Documents\\Data\\SPARC\\EstimatedSignal_qvec.txt'

Load data to use for prediction.

In [ ]:
source = 'SPARC_20'
gtab, data, voxelSize = preprocess_SPARC(dataPath[source])
data = data[:, :, 0, :]  # Remove singleton dimension

Load test data. The resulting gradient table (gtab_test) is identical to that you get by reverse engineering from the provided q-values, except that it includes b0.

In [ ]:
_, data_GS, _ = preprocess_SPARC(dataPath['SPARC_GS'])
data_GS = data_GS[:, :, 0, 1:]  # Remove b0 and singleton dimension
data_GS = data_GS.transpose((1, 0, 2))  # Transpose to get same shape as other data

Would be simpler to use the gradient table from the gold standard if it hadn't contained b0.

In [ ]:
def load_q_test(path):
    with open(path, 'r') as f:
        out = [line.split(' ') for line in f.readlines()]
    return np.array(out, dtype=float)

def gtab_from_q(gtab, q):
    small_delta = gtab.small_delta
    big_delta = gtab.big_delta
    tau = (big_delta - small_delta/3) * 1e-3
    q_magnitude = np.sqrt(np.sum(q_test ** 2, 1))
    bvals = (2*np.pi*q_magnitude) ** 2 * tau
    bvecs = q_test / q_magnitude[:, None]
    return gradient_table(bvals=bvals, bvecs=bvecs, small_delta=small_delta, big_delta=big_delta)

q_test = load_q_test(q_test_path)
gtab_test = gtab_from_q(gtab, q_test)

In [ ]:
lmbda=None
handler = DataHandler(gtab, data, qMagnitudeTransform=log_q_squared,
                      voxelSize=voxelSize[0:2], box_cox_lambda=lmbda)
handlerPred = DataHandler(gtab_test, data=None, spatial_shape=data_GS.shape[0:2],
                          qMagnitudeTransform=log_q_squared, voxelSize=voxelSize[0:2], box_cox_lambda=lmbda)


In [ ]:
spatialLengthScale = 5
bValLengthScale = 3

kernel = (GPy.kern.RBF(input_dim=1, active_dims=[0],
                       variance=1,
                       lengthscale=spatialLengthScale) *
          GPy.kern.RBF(input_dim=1, active_dims=[1],
                       variance=1,
                       lengthscale=spatialLengthScale) *
          GPy.kern.RBF(input_dim=1, active_dims=[2],
                            variance=1,
                            lengthscale=bValLengthScale) *
          GPy.kern.LegendrePolynomial(
             input_dim=3,
             coefficients=np.array((2, 0.5, 0.05)),
             orders=(0, 2, 4),
             active_dims=(3, 4, 5)))

kernel.parts[0].variance.fix(value=1)
kernel.parts[1].variance.fix(value=1)
kernel.parts[2].variance.fix(value=1)

grid_dims = [[0], [1], [2, 3, 4, 5]]

model = Model(handler, kernel, data_handler_pred=handlerPred, grid_dims=grid_dims, verbose=False)

In [ ]:
np.random.seed(0)
model.train(restarts=True)

print(model.GP_model)
print("\nLegendre coefficients: \n{}".format(model.GP_model.mul.LegendrePolynomial.coefficients))

In [ ]:
mu = model.predict(compute_var=False)

pred = {'GP': model.data_handler_pred.untransform(mu)}

## Reference prediction with dipy

In [ ]:
import dipy.reconst.dti as dti

tenmodel = dti.TensorModel(gtab)
tenfit = tenmodel.fit(data)

pred['DTI'] = tenfit.predict(gtab_test)

In [ ]:
import dipy.reconst.mapmri as mapmri

map_model = mapmri.MapmriModel(gtab, positivity_constraint=False)
mapfit = map_model.fit(data)

pred['MAP'] = mapfit.predict(gtab_test)

## Evaluation

In [ ]:
idx = 50 * np.arange(9)
f, axs = plt.subplots(len(idx), 4)
f.set_figheight(len(idx)*2)
f.set_figwidth(10)
for i, index in enumerate(idx):
    axs[i, 0].imshow(data_GS[:, :, index], vmin=0, vmax=1, cmap="GnBu_r")
    axs[i, 0].axis('off')
    axs[i, 0].set_title('Ground truth,\nb = {}'.format(np.round(gtab_test.bvals[index])))
    axs[i, 1].imshow(pred['GP'][:, :, index], vmin=0, vmax=1, cmap="GnBu_r")
    axs[i, 1].axis('off')
    axs[i, 1].set_title('GP prediction, \nb = {}'.format(np.round(gtab_test.bvals[index])))
    axs[i, 2].imshow(pred['DTI'][:, :, index], vmin=0, vmax=1, cmap="GnBu_r")
    axs[i, 2].axis('off')
    axs[i, 2].set_title('DTI prediction, \nb = {}'.format(np.round(gtab_test.bvals[index])))
    axs[i, 3].imshow(pred['MAP'][:, :, index], vmin=0, vmax=1, cmap="GnBu_r")
    axs[i, 3].axis('off')
    axs[i, 3].set_title('MAP-MRI prediction, \nb = {}'.format(np.round(gtab_test.bvals[index])))

plt.tight_layout()

In [ ]:
def get_SPARC_metrics(gtab, target, pred):
    bvals_in_range = (gtab.bvals <= 3500)  # 3500 instead of 3000 just to avoid round-off problems
    lowIdx = np.nonzero(bvals_in_range)
    highIdx = np.nonzero(np.invert(bvals_in_range))
    NMSE_low = compute_NMSE(target[:, :, lowIdx], pred[:, :, lowIdx])
    NMSE_high = compute_NMSE(target[:, :, highIdx], pred[:, :, highIdx])
    NMSE_all = compute_NMSE(target, pred)
    print("NMSE low: {}\nNMSE high: {}\nNMSE all: {}".format(NMSE_low, NMSE_high, NMSE_all))
    return NMSE_low, NMSE_high, NMSE_all

def compute_NMSE(target, pred):
    target = target.flatten()
    pred = pred.flatten()
    return np.mean(((target - pred)/target) **2)

for key, value in pred.items():
    print("\n{} model:".format(key))
    get_SPARC_metrics(gtab_test, data_GS, value)